<a href="https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/scripts/image_extraction_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire Event Capture and Visualization using Sentinel-2 Imagery

This notebook processes wildfire data using Sentinel-2 imagery. It masks clouds, generates rectangular areas around fire points(~1km), and creates URLs for visualized images.

## Key Functions

- **`earth_engine_init(project_id)`**: Authenticates and initializes the Earth Engine.
- **`mask_s2_clouds(image)`**: Masks clouds in Sentinel-2 images.
- **`load_fire_data(csv_path)`**: Loads fire data from a CSV file.
- **`fetch_image_collection(longitude, latitude, start_date, end_date)`**: Fetches an image collection for a fire event.
- **`generate_image_url(collection, rectangle)`**: Generates a visualization URL for a clipped image.
- **`process_fire_events(fire_df)`**: Processes fire data and retrieves image URLs.
- **`download_images(img_urls)`**: Downloads images from generated URLs.

## Usage

1. Set the project ID and CSV paths in the `main()` function.
2. Run `main()` to process the data and download images.


In [ ]:
import ee
import pandas as pd

# Authenticate and initialize Earth Engine
def earth_engine_init(project_id):
    ee.Authenticate()
    ee.Initialize(project=project_id)

# Function to mask clouds using the Sentinel-2 QA60 band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load the CSV file containing fire data
def load_fire_data(csv_path):
    return pd.read_csv(csv_path)

# Generate a rectangular area around fire points
def create_rectangle(longitude, latitude, buffer_size=0.015):
    coords = [
        [longitude - buffer_size, latitude - buffer_size],  # Bottom-left
        [longitude + buffer_size, latitude - buffer_size],  # Bottom-right
        [longitude + buffer_size, latitude + buffer_size],  # Top-right
        [longitude - buffer_size, latitude + buffer_size],  # Top-left
        [longitude - buffer_size, latitude - buffer_size]   # Closing the rectangle
    ]
    return ee.Geometry.Polygon(coords)

# Get image collection for a given fire event
def fetch_image_collection(longitude, latitude, start_date, end_date):
    rectangle = create_rectangle(longitude, latitude)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(rectangle)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .map(mask_s2_clouds)
                  .limit(40))
    return collection, rectangle

# Generate a URL for the clipped image
def generate_image_url(collection, rectangle):
    if collection.size().getInfo() > 0:
        # Select RGB bands (B4 = Red, B3 = Green, B2 = Blue)
        image = collection.median().select(['B4', 'B3', 'B2'])

        # Clip the image to the specific polygon (area of interest)
        clipped_image = image.clip(rectangle)

        # Generate a visualization URL for the clipped image
        url = clipped_image.getThumbURL({
            'min': 0, 'max': 0.5,  # Adjust the min/max for better visualization
            'dimensions': 512,
            'region': rectangle,
            'format': 'png'
        })
        return url
    return None

# Process each fire event and save image URLs
def process_fire_events(fire_df):
    print("Processing Fire Data...")
    image_urls = []

    for index, row in fire_df.iterrows():
        latitude = row['LATITUDE']
        longitude = row['LONGITUDE']

        # Convert acquisition date to 'YYYY-MM-DD' format
        start_date = pd.to_datetime(row['ACQ_DATE']).strftime('%Y-%m-%d')
        end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

        # Prepare the image collection and rectangle
        collection, rectangle = fetch_image_collection(longitude, latitude, start_date, end_date)

        # Generate URL for the clipped image
        url = generate_image_url(collection, rectangle)

        if url:
            image_urls.append({'latitude': latitude, 'longitude': longitude, 'url': url})

    print("Fire Data Processing Complete.")
    return image_urls

# Save image URLs to a CSV file
def save_to_csv(image_urls, output_csv_path):
    # Convert the list of image URLs to a pandas DataFrame
    image_urls_df = pd.DataFrame(image_urls)

    # Save the DataFrame to a CSV file
    image_urls_df.to_csv(output_csv_path, index=False)
    print(f"Image URLs saved to {output_csv_path}")

# Function to download and save images
def download_images(img_urls):
    for img in img_urls:
        try:
            response = requests.get(img['url'], stream=True)
            response.raise_for_status()  # Check for HTTP errors

            # Generate a filename based on latitude and longitude
            filename = f"downloaded_images/image_{img['latitude']}_{img['longitude']}.png"

            # Save the image
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Failed to download {img['url']}: {e}")


def main():
    # Modify as needed
    project_id = 'sdp-wildfire-detection-aze'
    csv_path = '/content/drive/MyDrive/fire_archive_M-C61_519848.csv'
    output_csv_path = '/content/image_urls.csv'

    os.makedirs('downloaded_images', exist_ok=True)

    earth_engine_init(project_id)
    fire_data = load_fire_data(csv_path)

    image_urls = process_fire_events(fire_data)

    download_images(image_urls)

main()


The command `!zip -r my_folder.zip my_folder` creates a compressed ZIP archive of the folder `my_folder`. This will be helpful for downloading the extracted images to your local machine in a single file.

In [ ]:
!zip -r my_folder.zip my_folder